# Installs, Imports, Drive Connection, WandB Connection

##### Installs

In [1]:
!pip install transformers
!pip install datasets
!pip install -U PyYAML
!pip install wandb

     |████████████████████████████████| 2.5 MB 38.0 MB/s 
     |████████████████████████████████| 895 kB 58.7 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 262 kB 32.3 MB/s 
     |████████████████████████████████| 243 kB 53.2 MB/s 
     |████████████████████████████████| 118 kB 47.1 MB/s 
     |████████████████████████████████| 636 kB 25.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.8 MB 30.5 MB/s 
     |████████████████████████████████| 97 kB 5.5 MB/s 
     |████████████████████████████████| 170 kB 40.8 MB/s 
     |████████████████████████████████| 138 kB 50.6 MB/s 
     |████████████████████████████████| 133 kB 46.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 62 kB 653 kB/s 
  Created wheel for subprocess32: filename=su

##### Imports

In [9]:
from pathlib import Path
from datasets import load_dataset

##### Drive Connection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Get Configuration

In [13]:
import configparser

config = configparser.ConfigParser()
config.read('/content/drive/MyDrive/Thesis/BookSuccessPredictor/config.ini')

['/content/drive/MyDrive/Thesis/BookSuccessPredictor/config.ini']

##### WandB Connection

In [4]:
# saves our models to artifacts in WandB
import wandb
%env WANDB_LOG_MODEL=true
%env WANDB_PROJECT=goodreads_pretrained_models

env: WANDB_LOG_MODEL=true
env: WANDB_PROJECT=goodreads_pretrained_models


In [5]:
wandb.login(key = config['WandB']['api_key'])

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Dataset Generator

### Load Text Data

#### goodreads_maharjan

For Within-task pre-training

In [24]:
base_path = Path(config['Datasets']['goodreads_maharjan_path'])
dataset = load_dataset(str(base_path / 'goodreadsnered.py'))

Dataset good_reads_practice_dataset downloaded and prepared to /root/.cache/huggingface/datasets/good_reads_practice_dataset/main_domain/1.1.0/2d3e0c5e46b10ced55eb2fed138958ef5f14854c7f2edf2bdfde1d44485d9c39. Subsequent calls will reuse this data.


#### goodreads_guarro

For In-domain pre-training

In [ ]:
base_path = Path(config['Datasets']['goodreads_guarro_path'])
dataset = load_dataset(str(base_path / 'goodreads_guarro_loading_script.py'))

### Tokenization

# Pre Training

## BERT 512

In [ ]:
# Name our project from WandB
%env WANDB_PROJECT=goodreads_success_predictor

env: WANDB_PROJECT=goodreads_success_predictor


In [ ]:
chunked_encoded_dataset = chunked_encoded_dataset.remove_columns(['book_title', 'genre', 'success_label'])

In [ ]:
chunked_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 14074
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 9872
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 9712
    })
})

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', additional_special_tokens = ['[CHARACTER]'])

from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(28997, 768)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
from transformers import Trainer, TrainingArguments

num_epoch = 5

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    "bert-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    report_to = "wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=chunked_encoded_dataset['train'],
    eval_dataset=chunked_encoded_dataset['validation']
)

trainer.train()

wandb.finish()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,2.822000,2.406996,209.843500,48.374000
2,2.315700,1.998168,209.853600,48.372000
3,2.169900,1.955170,209.779000,48.389000


train/loss,2.1699
train/learning_rate,0.0
train/epoch,3.0
train/global_step,5448
_runtime,4256
_timestamp,1620231928
_step,13
eval/loss,1.95517
eval/runtime,209.779
eval/samples_per_second,48.389
train/train_runtime,4256.4681


train/loss,█▆▆▄▃▃▂▁▁▁
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/epoch,▁▂▂▃▃▄▅▅▅▆▇▇██
train/global_step,▁▂▂▃▃▄▅▅▅▆▇▇██
_runtime,▁▂▂▃▃▄▄▅▅▆▇▇██
_timestamp,▁▂▂▃▃▄▄▅▅▆▇▇██
_step,▁▂▂▃▃▄▄▅▅▆▆▇▇█
eval/loss,█▂▁
eval/runtime,▇█▁
eval/samples_per_second,▂▁█
train/train_runtime,▁


## DistilBert 512

In [ ]:
# Name our project from WandB
%env WANDB_PROJECT=goodreads_success_predictor

env: WANDB_PROJECT=goodreads_success_predictor


In [ ]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased', additional_special_tokens = ['[CHARACTER]'])

from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(30523, 768)

In [ ]:
chunked_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 386820
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
from transformers import Trainer, TrainingArguments

num_epoch = 1

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    "distilbert-mlm-guarro_goodreads",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size = 16,
    num_train_epochs = 1,
    # per_device_eval_batch_size=32,
    weight_decay=0.01,
    report_to = "wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=chunked_encoded_dataset['train']
)

trainer.train()

wandb.finish()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: token_type_ids.
***** Running training *****
  Num examples = 386820
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24177
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: lucaguarro (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss


Saving model checkpoint to distilbert-mlm-guarro_goodreads/checkpoint-500
Configuration saved in distilbert-mlm-guarro_goodreads/checkpoint-500/config.json
Model weights saved in distilbert-mlm-guarro_goodreads/checkpoint-500/pytorch_model.bin
Saving model checkpoint to distilbert-mlm-guarro_goodreads/checkpoint-1000
Configuration saved in distilbert-mlm-guarro_goodreads/checkpoint-1000/config.json
Model weights saved in distilbert-mlm-guarro_goodreads/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to distilbert-mlm-guarro_goodreads/checkpoint-1500
Configuration saved in distilbert-mlm-guarro_goodreads/checkpoint-1500/config.json
Model weights saved in distilbert-mlm-guarro_goodreads/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to distilbert-mlm-guarro_goodreads/checkpoint-2000
Configuration saved in distilbert-mlm-guarro_goodreads/checkpoint-2000/config.json
Model weights saved in distilbert-mlm-guarro_goodreads/checkpoint-2000/pytorch_model.bin
Saving model ch

## ROBERTA 512

In [ ]:
chunked_encoded_dataset = chunked_encoded_dataset.remove_columns(['book_title', 'genre', 'success_label'])

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', additional_special_tokens = ['[CHARACTER]'])

from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('roberta-base')
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Embedding(50266, 768)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
from transformers import Trainer, TrainingArguments

num_epoch = 5

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    "roberta-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    num_train_epochs = 5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    report_to = "wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=chunked_encoded_dataset['train'],
    eval_dataset=chunked_encoded_dataset['validation']
)

trainer.train()

wandb.finish()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,1.939800,1.737092,238.022700,42.731000
2,1.858600,1.690681,237.600900,42.807000
3,1.827700,1.659989,237.420900,42.840000
4,1.772800,1.647067,237.410500,42.841000
5,1.755300,1.646039,237.429100,42.838000


train/loss,1.7553
train/learning_rate,0.0
train/epoch,5.0
train/global_step,9115
_runtime,7951
_timestamp,1620353649
_step,23
eval/loss,1.64604
eval/runtime,237.4291
eval/samples_per_second,42.838
train/train_runtime,7950.1742


train/loss,█▅▅▄▄▄▃▃▂▃▂▂▂▁▁▁▁▁
train/learning_rate,██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇████
_runtime,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇███
_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
eval/loss,█▄▂▁▁
eval/runtime,█▃▁▁▁
eval/samples_per_second,▁▆███
train/train_runtime,▁


## BIGBIRD 2048

In [ ]:
chunked_encoded_dataset = chunked_encoded_dataset.remove_columns(['book_title', 'genre', 'success_label'])

ValueError: ignored

In [ ]:
chunked_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 3697
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 2590
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 2547
    })
})

In [ ]:
from transformers import BigBirdTokenizer
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base', additional_special_tokens = ['[CHARACTER]'])

from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('google/bigbird-roberta-base')
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(50359, 768)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('google/bigbird-roberta-base')
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(50359, 768)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
from transformers import Trainer, TrainingArguments


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    "bigbird-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    report_to = "wandb",
    gradient_accumulation_steps = 4
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=chunked_encoded_dataset['train'],
    eval_dataset=chunked_encoded_dataset['validation']
)
# gradient_accumulation_steps 
trainer.train()

wandb.finish()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
0,2.539500,1.980498,1312.773900,1.973000
1,2.140800,1.918812,1367.968400,1.893000
2,2.071500,1.892684,1342.998400,1.929000


train/loss,2.0715
train/learning_rate,0.0
train/epoch,3.0
train/global_step,2772
_runtime,13055
_timestamp,1620279957
_step,8
eval/loss,1.89268
eval/runtime,1342.9984
eval/samples_per_second,1.929
train/train_runtime,13054.975


train/loss,█▃▂▂▁
train/learning_rate,█▆▅▃▁
train/epoch,▁▂▃▄▅▆▇██
train/global_step,▁▂▃▄▅▆▇██
_runtime,▁▃▃▄▅▆▇██
_timestamp,▁▃▃▄▅▆▇██
_step,▁▂▃▄▅▅▆▇█
eval/loss,█▃▁
eval/runtime,▁█▅
eval/samples_per_second,█▁▄
train/train_runtime,▁


In [ ]:
wandb.finish()

## ELECTRA 512

https://towardsdatascience.com/understanding-electra-and-training-an-electra-language-model-3d33e3a9660d

In [ ]:
# Name our project fro WandB
%env WANDB_PROJECT=goodreads_success_predictor

env: WANDB_PROJECT=goodreads_success_predictor


In [ ]:
chunked_encoded_dataset = chunked_encoded_dataset.remove_columns(['book_title', 'genre', 'success_label'])